In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
import string
import csv

In [2]:
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
# loading data

In [15]:

X_train= pd.read_csv('X_train.csv')
y_train= pd.read_csv('y_train.csv')

X_val= pd.read_csv('X_val.csv')
y_val= pd.read_csv('y_val.csv')

X_test= pd.read_csv('X_test.csv')
y_test= pd.read_csv('y_test.csv')

In [16]:
X_train=X_train['selftext'].apply(lambda x: str(x)).values
X_val=X_val['selftext'].apply(lambda x: str(x)).values
X_test=X_test['selftext'].apply(lambda x: str(x)).values

y_train=y_train['score'].values
y_val=y_val['score'].values
y_test=y_test['score'].values

In [17]:
#preprocessing:
# lowercases, tokenizes
text_vectorization= layers.TextVectorization(
    max_tokens=20000,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='multi_hot',
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False,
    encoding='utf-8',
    name='vectorization'
)


text_vectorization.adapt(X_train)

In [18]:
type(X_train)

numpy.ndarray

In [34]:
def get_model(max_tokens=20000, hidden_dim=256):
    text_input=keras.Input(shape=(),dtype='string')
    vectorized_text=text_vectorization(text_input)
    x=layers.Dense(hidden_dim, activation='tanh')(vectorized_text)
    x=layers.Dropout(0.5)(x)
    outputs=layers.Dense(1)(x)
    model=keras.Model(text_input,outputs)
    model.compile(optimizer='Adam', loss='mse', metrics='mse')
    return model



In [35]:
model=get_model()
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None,)]                 0         
                                                                 
 vectorization (TextVectori  (None, 20000)             0         
 zation)                                                         
                                                                 
 dense_8 (Dense)             (None, 256)               5120256   
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 5120513 (19.53 MB)
Trainable params: 5120513 (19.53 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [36]:
callbacks=keras.callbacks.ModelCheckpoint('keras_bag_words.keras', save_best_only=True)
model.fit(
    X_train, y_train,
    epochs=10,#increase number of epochs
    validation_data=(X_val, y_val),
    callbacks=callbacks
    )


Epoch 1/10
800/800 [==============================] - 59s 73ms/step - loss: 1598500.0000 - mse: 1598500.0000 - val_loss: 909474.3125 - val_mse: 909474.3125
Epoch 2/10
800/800 [==============================] - 62s 77ms/step - loss: 1594559.2500 - mse: 1594559.2500 - val_loss: 906015.3125 - val_mse: 906015.3125
Epoch 3/10
800/800 [==============================] - 62s 77ms/step - loss: 1590877.6250 - mse: 1590877.6250 - val_loss: 903099.3750 - val_mse: 903099.3750
Epoch 4/10
800/800 [==============================] - 60s 75ms/step - loss: 1587276.1250 - mse: 1587276.1250 - val_loss: 900667.1250 - val_mse: 900667.1250
Epoch 5/10
800/800 [==============================] - 66s 82ms/step - loss: 1583640.8750 - mse: 1583640.8750 - val_loss: 898842.6875 - val_mse: 898842.6875
Epoch 6/10
800/800 [==============================] - 59s 73ms/step - loss: 1580014.1250 - mse: 1580014.1250 - val_loss: 897139.5000 - val_mse: 897139.5000
Epoch 7/10
800/800 [==============================] - 62s 77ms/s

In [39]:

MSE_train = model.evaluate(X_train, y_train, verbose=0)[0]
MSE_val = model.evaluate(X_val, y_val, verbose=0)[0]
MSE_test = model.evaluate(X_test, y_test, verbose=0)[0]

print('MSE for train set', MSE_train)
print('MSE for val set', MSE_val)
print('MSE for test set', MSE_test)



MSE for train set 1562336.875
MSE for val set 891054.125
MSE for test set 1288276.75
